In [1]:
pip install mediapipe

   ---------------------------------------- 0.0/50.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/50.8 MB 3.6 MB/s eta 0:00:14
   ---------------------------------------- 0.2/50.8 MB 2.5 MB/s eta 0:00:20
   ---------------------------------------- 0.6/50.8 MB 4.0 MB/s eta 0:00:13
    --------------------------------------- 0.8/50.8 MB 4.2 MB/s eta 0:00:13
    --------------------------------------- 1.1/50.8 MB 4.8 MB/s eta 0:00:11
    --------------------------------------- 1.1/50.8 MB 3.9 MB/s eta 0:00:13
   - -------------------------------------- 1.3/50.8 MB 4.1 MB/s eta 0:00:13
   - -------------------------------------- 1.6/50.8 MB 4.5 MB/s eta 0:00:11
   - -------------------------------------- 1.9/50.8 MB 4.6 MB/s eta 0:00:11
   - -------------------------------------- 2.1/50.8 MB 4.8 MB/s eta 0:00:11
   - -------------------------------------- 2.4/50.8 MB 5.0 MB/s eta 0:00:10
   -- ------------------------------------- 2.9/50.8 MB 5.4 MB/s eta 0:00:09
   -- 

In [3]:
pip install screen-brightness-control

Note: you may need to restart the kernel to use updated packages.


In [1]:
# Importing Libraries 
import cv2 
import mediapipe as mp 
from math import hypot 
import screen_brightness_control as sbc 
import numpy as np 


In [3]:
# Initializing the Model 
mpHands = mp.solutions.hands 
hands = mpHands.Hands( 
	static_image_mode=False, 
	model_complexity=1, 
	min_detection_confidence=0.75, 
	min_tracking_confidence=0.75, 
	max_num_hands=2) 

Draw = mp.solutions.drawing_utils 


In [ ]:
# Start capturing video from webcam 
cap = cv2.VideoCapture(0) 

while True: 
	# Read video frame by frame 
	_,frame = cap.read() 
	
	#Flip image 
	frame=cv2.flip(frame,1) 
	
	# Convert BGR image to RGB image 
	frameRGB = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB) 
	
	# Process the RGB image 
	Process = hands.process(frameRGB) 
	
	landmarkList = [] 
	# if hands are present in image(frame) 
	if Process.multi_hand_landmarks: 
		# detect handmarks 
		for handlm in Process.multi_hand_landmarks: 
			for _id,landmarks in enumerate(handlm.landmark): 
				# store height and width of image 
				height,width,color_channels = frame.shape 
				
				# calculate and append x, y coordinates 
				# of handmarks from image(frame) to lmList 
				x,y = int(landmarks.x*width),int(landmarks.y*height)			 
				landmarkList.append([_id,x,y]) 
			
			# draw Landmarks 
			Draw.draw_landmarks(frame,handlm,mpHands.HAND_CONNECTIONS) 
	
	# If landmarks list is not empty 
	if landmarkList != []: 
		# store x,y coordinates of (tip of) thumb 
		x_1,y_1 = landmarkList[4][1],landmarkList[4][2] 
		
		# store x,y coordinates of (tip of) index finger 
		x_2,y_2 = landmarkList[8][1],landmarkList[8][2] 
		
		# draw circle on thumb and index finger tip 
		cv2.circle(frame,(x_1,y_1),7,(0,255,0),cv2.FILLED) 
		cv2.circle(frame,(x_2,y_2),7,(0,255,0),cv2.FILLED) 
		
		# draw line from tip of thumb to tip of index finger 
		cv2.line(frame,(x_1,y_1),(x_2,y_2),(0,255,0),3) 
		
		# calculate square root of the sum 
		# of squares of the specified arguments. 
		L = hypot(x_2-x_1,y_2-y_1) 
		
		# 1-D linear interpolant to a function 
		# with given discrete data points 
		# (Hand range 15 - 220, Brightness range 0 - 100), 
		# evaluated at length. 
		b_level = np.interp(L,[15,220],[0,100]) 
	
		# set brightness 
		sbc.set_brightness(int(b_level)) 

	# Display Video and when 'q' is entered, 
	# destroy the window 
	cv2.imshow('Image', frame) 
	if cv2.waitKey(1) & 0xff == ord('q'): 
		break


C:\Users\Dell\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
